In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import sys
sys.path.append('../')

from geometry import Geometry
from forcing import Forcing
from layer import LayerModel

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=2)

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
x1,x2 = 245.4-360,246.2-360
y1,y2 = -75.13,-74.85

cols = ['teal','darkgoldenrod','firebrick']

days = np.arange(1,30)
yy = 1966

ress = ['0.5','1.0','2.0']
lws = [2,3,2]

In [ ]:
def addvar(i,j,var,r,title):
    ax[i,j].plot(days,var,c=cols[r],label=f'{ress[r]}km',lw=lws[r])
    ax[i,j].set_ylabel(title)
    

In [ ]:
mpl.rc('figure.subplot',left=.1,right=.99,top=.99,bottom=.1,wspace=.3,hspace=.1)

fig,ax = plt.subplots(3,2,sharex=True,figsize=(7,6))

for r,res in enumerate(ress):
    MELT = np.nan*np.zeros(len(days))
    Mkw  = np.nan*np.zeros(len(days))
    Dav = np.nan*np.zeros(len(days))
    Tav = np.nan*np.zeros(len(days))
    Sav = np.nan*np.zeros(len(days))
    Etot = np.nan*np.zeros(len(days))
    FT  = np.nan*np.zeros(len(days))
    for d,day in enumerate(days):
        try:
            ds = xr.open_dataset(f'../../results/spinup/CrossDots_{res}_mitgcm_{yy}_{yy}_{day:03.0f}.nc')
            dx = (ds.x[1]-ds.x[0]).values
            MELT[d] = xr.where(ds.tmask==1,ds.melt,np.nan).mean()
            melt = ds.melt.copy()
            melt = xr.where(ds.lon<x2,melt,np.nan)
            melt = xr.where(ds.lat<y2,melt,np.nan)

            Mkw[d] = xr.where(ds.tmask==1,melt,np.nan).mean()
            Dav[d]  = xr.where(ds.tmask==1,ds.D,np.nan).mean()
            Tav[d]  = (ds.tmask*ds.D*ds.T).sum()/(ds.tmask*ds.D).sum()
            Sav[d]  = (ds.tmask*ds.D*ds.S).sum()/(ds.tmask*ds.D).sum()
            Etot[d] = 1e-3*(ds.tmask*ds.entr*dx*dx).sum()/(3600*24*365.25) #mSv
            
            FT[d] = 360*xr.where(ds.tmask==1,ds.D,np.nan).sum()/xr.where(ds.tmask==1,ds.entr,np.nan).sum().values
            ds.close()
        except:
            pass
    addvar(0,0,MELT,r,'Average melt [m/yr]')
    addvar(0,1,Mkw,r,'Kohler West melt [m/yr]')
    addvar(1,0,Tav,r,'Average temp [degC]')
    addvar(1,1,Sav,r,'Average saln [psu]')
    addvar(2,0,Dav,r,'Average thickness [m]')
    addvar(2,1,Etot,r,'Overturning [mSv]')

#ax[0,0].plot(days,7+0*days,c='.5',lw=.5)
ax[0,0].scatter(30,7,50,color='.5',marker='d')
ax[2,1].scatter(30,170,50,color='.5',marker='d')
    
ax[0,0].legend(loc='upper right')
fig.supxlabel('Model days',fontsize=10);

plt.savefig('../../figures/draftplot_spinup.png',dpi=450)